# Coref resolution

In [6]:
import pickle

# load saved model
with open('allen_coref' , 'rb') as f:
    predictor = pickle.load(f)

text = "A cat is walking on a sunny day. It tripped and fell on a stone. It died."

prediction = predictor.predict(document=text)  # get prediction
print("Clsuters:-")
for cluster in prediction['clusters']:
    print(cluster)  # list of clusters (the indices of spaCy tokens)
# Result: [[[0, 3], [26, 26]], [[34, 34], [50, 50]]]
print('\n\n') #Newline

coref_text = predictor.coref_resolved(text)

print('Coref resolved: ', coref_text)  # resolved text
# Result: Joseph Robinette Biden Jr. is an American politician who is the 46th and

Clsuters:-
[[0, 1], [9, 9], [17, 17]]



Coref resolved:  A cat is walking on a sunny day. A cat tripped and fell on a stone. A cat died.


# SAO parsing

In [3]:
# !pip install textacy

In [7]:
from subject_verb_object_extract import findSVOs, nlp
# coref_text = "A cat is walk rapidly on a sunny day. A cat fall on a stone. A cat dead"
tokens = nlp(coref_text)
svos = findSVOs(tokens)
print(coref_text)
print(svos)

A cat is walking on a sunny day. A cat tripped and fell on a stone. A cat died.
[('A cat', 'walking'), ('A cat', 'died')]


In [4]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

actions = ['dead', 'fall', 'hurt', 'idle', 'jump', 'run', 'slide', 'walk']

In [27]:
# !pip install pygame

In [9]:
import pygame
import glob
 
SIZE = WIDTH, HEIGHT = 600, 600 #the width and height of our screen
FPS = 9 #Frames per second
 
class MySprite(pygame.sprite.Sprite):
    def __init__(self):
        super(MySprite, self).__init__()
        
        self.images = []
        # self.images = [pygame.image.load(img) for img in glob.glob("pygame\\cat\\*.png")]
        for svo in svos:
            action = stemmer.stem(svo[-1])
            print(action)

            self.images.extend([pygame.image.load(img) for img in glob.glob("pygame\\cat\\" + action + "*.png")])
            
        print(self.images)
        self.index = 0
        self.rect = pygame.Rect(5, 5, 150, 198)
 
    def update(self):
        if self.index >= len(self.images):
            self.index = 0
        self.image = self.images[self.index]
        self.index += 1
 
def main():
    pygame.init()
    screen = pygame.display.set_mode(SIZE)
    pygame.display.set_caption("Trace")
    my_sprite = MySprite()
    my_group = pygame.sprite.Group(my_sprite)
    clock = pygame.time.Clock()
    loop = 1
    while loop:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                loop = 0
 
        my_group.update()
        screen.fill((0,0,0))
        my_group.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
    pygame.quit()
 
if __name__ == '__main__':
    main()

walk
die
[<Surface(542x474x32 SW)>, <Surface(542x474x32 SW)>, <Surface(542x474x32 SW)>, <Surface(542x474x32 SW)>, <Surface(542x474x32 SW)>, <Surface(542x474x32 SW)>, <Surface(542x474x32 SW)>, <Surface(542x474x32 SW)>, <Surface(542x474x32 SW)>, <Surface(542x474x32 SW)>, <Surface(556x504x32 SW)>, <Surface(556x504x32 SW)>, <Surface(556x504x32 SW)>, <Surface(556x504x32 SW)>, <Surface(556x504x32 SW)>, <Surface(556x504x32 SW)>, <Surface(556x504x32 SW)>, <Surface(556x504x32 SW)>, <Surface(556x504x32 SW)>, <Surface(556x504x32 SW)>]


# NER

In [45]:
import spacy 
import spacy.cli
# Download following once
#spacy.cli.download("en_core_web_sm")

In [46]:
from spacy import displacy

nlp = spacy.load('en_core_web_sm')

In [47]:
text = "Victoria was a major in CS. Victoria studies at MIT."
doc = nlp(text)

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(ents)
# now we use displaycy function on doc2
displacy.render(doc, style='ent', jupyter=True)

[('CS', 24, 26, 'GPE'), ('MIT', 48, 51, 'ORG')]
